# Import

In [11]:
import numpy as np

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from torch.utils.data import Dataset
from torchmetrics.classification import BinaryF1Score
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinarySpecificity
from torchmetrics.classification import BinaryAUROC
from torchmetrics.classification import BinaryAccuracy

 # DataSet & DataLoader preparation


## Dataset

In [12]:
class DatasetPTBXL(Dataset):

    def __init__(self, path,p_list,p_name):
        self.path = path
        self.len = len(os.listdir(self.path))
        self.target_list = p_list
        self.target_name = p_name
    def __len__(self):
        return self.len


    def __getitem__(self, idx):
        file = np.load(path + str(idx) + '.npy', allow_pickle=True).item()
        return torch.from_numpy(file['data']), torch.from_numpy(file['target'][ptlg_list.index(ptlg_name)])

## Inputs

In [13]:
path = '/home/narmarinad_12/Desktop/ECG/files_processed/'
ptlg_list = ['SR', 'AFIB', 'STACH', 'SARRH', 'SBRAD', 'PACE', 'SVARR', 'BIGU', 'AFLT', 'SVTAC', 'PSVT', 'TRIGU']
ptlg_name = 'SR'


## Split and DataLoad

In [16]:
train_dataset = DatasetPTBXL(path,ptlg_list,ptlg_name)

tmp_len = len(train_dataset)
train_size = int (0.8 * tmp_len) 
valid_size = int (0.1 * tmp_len)
test_size = tmp_len - train_size - valid_size

train_dataset, valid_dataset,test_dataset = random_split(train_dataset, [train_size, valid_size, test_size])                                                                       
batch_size = 500
train_dataset = DataLoader(train_dataset, batch_size, num_workers = 5, pin_memory = True ) 
valid_dataset = DataLoader(valid_dataset, batch_size, num_workers = 5, pin_memory = True )
test_dataset  = DataLoader(test_dataset , batch_size, num_workers = 5, pin_memory = True )

# Model

 



In [ ]:
class Block(nn.Module):
    
    def __init__(self, in_channels, out_channels, identity_downsample = None, stride=1):
        super(Block, self).__init__()

        self.resblock = nn.Sequential(nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
                      nn.BatchNorm1d(out_channels),
                      nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                      nn.BatchNorm1d(out_channels))

        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        
    def forward(self, x):
        identity = x
        x = self.resblock(x)
        #in case we need to double size of identity
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x += identity
        x = self.relu(x)
        return x

In [ ]:
#ResNet34
class RhythmECGClassification(nn.Module):
    
    def __init__(self, in_channels, num_classes,class_index,device):
        
        super(RhythmECGClassification, self).__init__()
        self.class_index = class_index
        self.in_channels = in_channels
        self.device = device
        self.layer0 = nn.Sequential(nn.Conv1d(in_channels, 64, kernel_size=7, stride=2, padding=3),
                                    nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(kernel_size=3, stride=2, padding=1))
        self.layer1 = self.create_layer(64, 64, stride=1)
        self.layer2 = self.create_layer(64, 128, stride=2)
        self.layer3 = self.create_layer(128, 128, stride=1)
        self.layer4 = self.create_layer(128, 256, stride=2)
        self.layer5 = self.create_layer(256, 256, stride=1)
        self.layer6 = self.create_layer(256, 512, stride=2)
        self.layer7 = self.create_layer(512, 512, stride=1)
        self.layer8 = nn.Sequential(nn.AdaptiveAvgPool1d(1), nn.Flatten())
        self.layer9 = nn.Sequential(nn.Dropout(0.2),nn.Linear(512, num_classes), nn.Sigmoid())
        
    #with class Block create layers
    def create_layer(self, in_channels, out_channels, stride):
        
        identity_downsample = None
        if stride != 1:
            identity_downsample = nn.Sequential(nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=2, padding=1), 
                                                nn.BatchNorm1d(out_channels))
        
        return nn.Sequential(to_device(Block(in_channels, out_channels, identity_downsample, stride),
                                       self.device),
                             to_device(Block(out_channels, out_channels),self.device))
        
        
    def forward(self, x):

        x = self.layer0(x)
  
        x = self.layer1(x)
        x = self.layer1(x)
        x = self.layer1(x)

        x = self.layer2(x)

        x = self.layer3(x)
        x = self.layer3(x)
        x = self.layer3(x)

        x = self.layer4(x)

        x = self.layer5(x)
        x = self.layer5(x)
        x = self.layer5(x)
        x = self.layer5(x)
        x = self.layer5(x)

        x = self.layer6(x)

        x = self.layer7(x)
        x = self.layer8(x)

        x = self.layer9(x)
        return x 
    
   
    #calculating batch loss
    def training_step(self,batch):
      signals,targets = batch
      preds = self((signals.float())) 
      loss = nn.BCELoss()
      return loss(preds, torch.unsqueeze(targets[:,self.class_index],1).float())
    

    #making prediction to count metrics
    def validation_step(self,batch):
      signals,targets = batch
      preds = self((signals.float()))
      return { 'batch_preds': preds.detach(), 'batch_targets': torch.unsqueeze(targets[:,self.class_index],1).float()}

# 3. Оценивание
    
    





In [ ]:
#calculating metrics of epoch, returning dict of metrics
def evaluate(model, valid_dataset,device):
  model.eval()
  metric_F1 = BinaryF1Score().to(device)
  metric_Rec = BinaryRecall().to(device)
  metric_Spec = BinarySpecificity().to(device)
  metric_AUC = BinaryAUROC(thresholds=None).to(device)
  metric_Acc = BinaryAccuracy().to(device)
  with torch.no_grad():
    for batch in valid_dataset:
      tmp = model.validation_step(batch)
      metric_F1(tmp['batch_preds'], tmp['batch_targets'] )
      metric_Rec(tmp['batch_preds'], tmp['batch_targets'] )
      metric_Spec(tmp['batch_preds'], tmp['batch_targets'] )
      metric_AUC(tmp['batch_preds'], tmp['batch_targets'] )
      metric_Acc(tmp['batch_preds'], tmp['batch_targets'] )


  
  return {'model_acc': metric_Acc.compute().item(),'model_F1score': metric_F1.compute().item(),'model_Recall': metric_Rec.compute().item(),
          'model_Specificity': metric_Spec.compute().item(), 'model_ROCAUC': metric_AUC.compute().item() }


In [ ]:
#ploting metrics/epoch
import matplotlib.pyplot as plt

def visual_res(history, epoch_num ):
  plt.title('Динамика обучения') 
  plt.xlabel('Эпоха')
  plt.ylabel('Метрика')
  
  
  # list_of_metrics содержит наборы значений метрик для построения графика метрика/номер эпохи
  plt.plot(list(range(epoch_num)), [x['model_F1score'] for x in history],label = 'F1')
  plt.plot(list(range(epoch_num)), [x['model_Recall'] for x in history],label = 'Recall')
  plt.plot(list(range(epoch_num)), [x['model_Specificity'] for x in history], label = 'Specificity')
  plt.plot(list(range(epoch_num)), [x['model_ROCAUC'] for x in history], label = 'AUC')
  plt.plot(list(range(epoch_num)), [x['model_acc'] for x in history], label = 'Acc')
  # plt.plot(list(range(epoch_num)), [x['epoch_loss'] for x in history], label = 'loss')
  plt.grid(True)
  plt.legend()
  #построение графиков

# 4. Использование GPU

In [ ]:

def get_default_device():
    if torch.cuda.is_available(): return torch.device('cuda')
    else: return torch.device('cpu')
 
def to_device(data, device):
  if isinstance(data, (list)): return [to_device(x, device) for x in data]
  return data.to(device, non_blocking=True)
  
#creating generator for batch to gpu
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)
        
device = get_default_device()
train_dataset = DeviceDataLoader(train_dataset, device)
valid_dataset = DeviceDataLoader(valid_dataset, device)
test_dataset = DeviceDataLoader(test_dataset, device)

# 5. Функция тренировки модели
    Сохраняем метрики и динамику функции потерь для каждой эпохи в переменной history.


In [ ]:
def fit(epoch_num, lr, model, gc, wd ,opt,device,key):
  
  patallogies = {0: 'sinus_rhythm', 1: 'atrial_fibrillation',2: 'sinus tachycardia',
                 3: 'sinus arrhythmia', 4: 'sinus_bradycardia'}

  opt = opt(model.parameters(), lr, weight_decay = wd )
  history = []
  #for every epoch we will save model in Model.dir
  #later we will chose one with better scores
  for epoch in range(epoch_num):
    torch.cuda.empty_cache()
    model.train()
    train_losses = 0
    for batch in train_dataset:
      loss = model.training_step(batch) 
      train_losses/=2
      train_losses+=loss.detach()
      loss.backward()
      nn.utils.clip_grad_value_(model.parameters(), gc)
      opt.step()
      opt.zero_grad()
      
    result = evaluate(model, valid_dataset,device)
    history.append(result)

    print('Epoch ', epoch,end = "")
    print(': train_loss = %.4f'%train_losses)

    PATH = f'/content/drive/My Drive/ColabNotebooks/ECG/Model1/{patallogies[key]}{epoch}.pth'
    torch.save(model.state_dict(), PATH)
  return history


def train_model(res, lr, epochs,  device,key_t, opt_func = torch.optim.Adam):

  history = []
  model = res
  history += fit(epochs, lr, model, gc, wd, opt_func,device,key_t)
  visual_res(history,epochs)
  
  return model

# 6. Тренировка модели


In [ ]:
lr = 0.1
epochs = 1
gc = 0.1
wd = 0.01
pat = 3

In [ ]:
res = to_device(RhythmECGClassification(12,1,pat,device), device)

In [ ]:
res = train_model(res, lr,epochs,device, pat)
eval = evaluate(res, test_dataset,device)
for keys in eval:
    print(keys, '=' ,eval[keys])

# 7. Результат. Загрузка модели.

In [ ]:
general_metrics = {}
general_metrics1 = {}
patallogies = {0: 'sinus_rhythm', 1: 'atrial_fibrillation',2: 'sinus tachycardia',
                 3: 'sinus arrhythmia', 4: 'sinus_bradycardia'}

In [ ]:
word = 'sinus_bradycardia1'
PATH = f'/content/drive/My Drive/ColabNotebooks/ECG/Model1/{word}.pth'
res = to_device(RhythmECGClassification(12,1,4,device), device)
res.load_state_dict(torch.load(PATH))
# print(the_model)
eval = evaluate(res, test_dataset,device)
for keys in eval:
    print(keys, '=' ,eval[keys])
general_metrics[word] = eval


model_acc = 0.9708113074302673
model_F1score = 0.0
model_Recall = 0.0
model_Specificity = 1.0
model_ROCAUC = 0.6421321630477905


In [ ]:
general_metrics

{'sinus_rhythm1': {'model_acc': 0.833371639251709,
  'model_F1score': 0.9030100107192993,
  'model_Recall': 0.9944018721580505,
  'model_Specificity': 0.2622779607772827,
  'model_ROCAUC': 0.8836913108825684},
 'atrial_fibrillation18': {'model_acc': 0.7692484259605408,
  'model_F1score': 0.370138019323349,
  'model_Recall': 1.0,
  'model_Specificity': 0.7524654865264893,
  'model_ROCAUC': 0.9929792284965515},
 'sinus tachycardia4': {'model_acc': 0.9820730686187744,
  'model_F1score': 0.7845304012298584,
  'model_Recall': 0.9161290526390076,
  'model_Specificity': 0.9845090508460999,
  'model_ROCAUC': 0.9910898208618164},
 'sinus arrhythmia5': {'model_acc': 0.9680533409118652,
  'model_F1score': 0.0,
  'model_Recall': 0.0,
  'model_Specificity': 1.0,
  'model_ROCAUC': 0.5116299986839294},
 'sinus_bradycardia1': {'model_acc': 0.9708113074302673,
  'model_F1score': 0.0,
  'model_Recall': 0.0,
  'model_Specificity': 1.0,
  'model_ROCAUC': 0.6421321630477905}}

# Тесты и итоговые метрики